In [2]:
import os
import re
import pandas as pd

In [16]:
whatsapp_file = 'WHATSAPP_CHAT_PATH'


# Whatsapp dosyasını okuyoruz
if os.path.exists(whatsapp_file): # dosya varlık kontrolü yapıyoruz
        dosya_data = open(whatsapp_file,'r', encoding="utf-8") # dosyayı okuma modunda ('r') açıyoruz
        whatsapp_icerik = dosya_data.read()

# Tarihi Alıyoruz
tarih_regex=re.compile(r'(\d+/\d+/\d+)') # tarih formatımıza uygun regex yazıyoruz
tarih=tarih_regex.findall(whatsapp_icerik)  # tarihleri regex ile alıp liste olarak değişkene atıyoruz

# Zamanı Alıyoruz
zaman_regex=re.compile(r'(\d{1,2}:\d{2})')
zaman=zaman_regex.findall(whatsapp_icerik)

# Kullanıcıyı Alıyoruz
kullanici_regex=re.compile(r'-(.*?):')
kullanici=kullanici_regex.findall(whatsapp_icerik)

# Mesajı Alıyoruz
mesaj_regex=re.compile(r'([^:]+):?$')
me_regex=re.compile(r"(\n)(?<=)(\d+/\d+/\d+)(.*)")
mess=me_regex.findall(whatsapp_icerik)
mesaj = [''.join(mesaj_regex.findall(''.join(msg))).strip() for msg in mess]

# Aldığımız tüm verileri bir arada zipliyoruz
data=[]
for w,x,y,z in zip(tarih,zaman,kullanici,mesaj):
    data.append([str(w),str(x),str(y),str(z)])
    
#print(data[0])


# DataFrame oluşturuyoruz
df=pd.DataFrame(data,columns=("Tarih","Zaman","Kullanici","Mesaj"))

# Mesajları temizliyoruz
df['Mesaj']=df['Mesaj'].str.replace('\'(.*?): ','')

# Tarih sütununu DataTime tipinde güncelliyoruz
df['Tarih'] = pd.to_datetime(df['Tarih'])

# Yıl sütununu Tarih'in yılı olarak set ediyoruz (oluşturuyoruz)
df['Yil']=df['Tarih'].dt.year

# Ay sütununu Tarih'in yılı olarak set ediyoruz (oluşturuyoruz)
df['Ay']=df['Tarih'].dt.month

# Gun sütununu Tarih'in günü olarak set ediyoruz (oluşturuyoruz)
df['Gun']=df['Tarih'].dt.day

# İstersek günün hangi saatinde atıldığını da alabiliriz
#df['Zaman'] = pd.to_datetime(df['Zaman'])
#df['Saat']=df.Zaman.apply(lambda x: x.hour)

# Mesajdan kelimeleri ayıklıyoruz
df['Kelimeler'] = df['Mesaj'].str.strip().str.split('[\W_]+')

# Toplam kelime sayısı
df['Kelime Sayisi'] = df['Kelimeler'].apply(len)

# Mesaj karakter sayisi
df['Mesaj Karakter Sayisi'] = df['Mesaj'].map(str).apply(len)-3

# Mesajdan medyaları alıyoruz
df['Medya']=df['Mesaj'].str.contains('<Media omitted>')

# Oluşturduğumuz DataFrame'i csv olarak dışa aktarıyoruz
df.to_csv("whatsapp.csv")